In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

classifier = pipeline("ner", model="stulcrad/CNEC_2_0_robeczech-base")
str_ = "Jste světa znalý muž a víte stejně dobře jako já , že souvislost mezi současnými krutostmi v Jihovýchodní Asii a tou novou bankovní pobočkou hned vedle obchoďáku Zátoka je přímá a bezprostřední ; byl z toho už vzteklý jak uvázaný pes , protože zájemci o hodiny mu úplně narušili jeho denní režim a on si nemohl po obědě ani zdřímnout ."
classifier(str_)

In [ ]:
str_[162:168]

In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

def combine_adjacent_name_tokens(results):
    """
    Spojí navazující tokeny jmen do celých jmen.
    
    Args:
        results: Seznam tokenů z NER modelu.
    
    Returns:
        Seznam slovníků obsahujících spojená jména.
    """
    # Typy tokenů pro jména
    person_types = ['pf', 'ps', 'pp', 'pm', 'pc', 'pd' ]
    
    combined_names = []
    current_name = None
    last_index = -1
    
    # Seřazení výsledků podle indexu
    sorted_results = sorted(results, key=lambda x: x['index'])
    
    for token in sorted_results:
        # Rozdělení entity na typ
        entity_type = token['entity'].split('-')[1] if '-' in token['entity'] else ""
        if entity_type in person_types:
            if current_name is None or token['index'] != last_index + 1:
                # Nové jméno
                if current_name:
                    combined_names.append(current_name)
                current_name = {
                    'start': token['start'],
                    'end': token['end']
                }
            else:
                # Přidání dalšího tokenu
                current_name['end'] = token['end']
            
            last_index = token['index']
    
    # Přidání posledního jména
    if current_name:
        combined_names.append(current_name)
        
    return combined_names

def extract_and_mark_names(text, results):
    """
    Ohraničí nalezená jména značkami @@ a ##.
    
    Args:
        text: Původní text.
        results: Výstupy z NER modelu.
    
    Returns:
        Text s označenými jmény.
    """
    
    combined = combine_adjacent_name_tokens(results)
    combined.sort(key=lambda x: x['start'])
    
    marked_text = ""
    last_end = 0
    
    # Přidání @@## k nalezeným jménům
    for name in combined:
        marked_text += text[last_end:name['start']]
        
        original_name = text[name['start']:name['end']]
        marked_text += "@@" + original_name + "##"
        
        last_end = name['end']
    
    marked_text += text[last_end:]
    
    return marked_text

def process_dataset_with_model(input_file, output_file):
    """
    Zpracuje dataset pomocí modelu a uloží označené kontexty do výstupního souboru.
    
    Args:
        input_file: Cesta k vstupnímu JSON souboru.
        output_file: Cesta k výstupnímu textovému souboru.
    """
    # Načtení modelu
    model = AutoModelForTokenClassification.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
    tokenizer = AutoTokenizer.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
    classifier = pipeline("ner", model=model, tokenizer=tokenizer)
    
    # Načtení datasetu
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Zpracování a uložení
    with open(output_file, 'w', encoding='utf-8') as f:
        for item in data:
            context = item["context"]
            results = classifier(context)
            marked_context = extract_and_mark_names(context, results)
            f.write(marked_context + '\n')

if __name__ == "__main__":
    input_file = "input_dataset.json"
    output_file = "hokuspokus.txt"
    process_dataset_with_model(input_file, output_file)

e:\GitHub\GPT-NER\gpt-ner-env-py38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
str = "Polovičatost modu vivendi mezi Izraelem a Bohem , placená zamlžováním boží pravdy a lidského hříchu , znevážením Boha i člověka , jitřila Ježíše tak , že se rozhodl nemlčet a handlování s Bohem skončit , ve skutečnosti však experiment s jeho všemohoucností vyostřit ."
model = AutoModelForTokenClassification.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
tokenizer = AutoTokenizer.from_pretrained("stulcrad/CNEC_2_0_robeczech-base")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
results = classifier(str)
print(results)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-gc', 'score': 0.99654895, 'index': 10, 'word': 'ĠIzraelem', 'start': 31, 'end': 39}, {'entity': 'B-pp', 'score': 0.9708674, 'index': 12, 'word': 'ĠBohem', 'start': 42, 'end': 47}, {'entity': 'B-pp', 'score': 0.9767138, 'index': 28, 'word': 'ĠBoha', 'start': 113, 'end': 117}, {'entity': 'B-pp', 'score': 0.9637986, 'index': 34, 'word': 'ĠJeÅ¾ÃŃÅ¡e', 'start': 138, 'end': 144}, {'entity': 'B-pp', 'score': 0.9630684, 'index': 48, 'word': 'ĠBohem', 'start': 188, 'end': 193}]
